In [69]:
from gensim.models import Word2Vec
import pandas as pd
import os
import numpy as np
import sys
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from sklearn.cluster import AgglomerativeClustering
from bokeh import palettes
from sqlalchemy import create_engine
import pickle

# Installation (if fast = True below):
# https://pypi.org/project/fitsne/0.1.10/
# I'm unsure if the fftw.org part (below) is needed, 
# or if the pip commands listed on the pypi link 
# are enough to get it working (and get it working on multiple cores).
# Download and execute:
# http://fftw.org/install/mac.html with commands below
# $./configure --enable-threads && make
# $ sudo make install
# You NEED to clone https://github.com/KlugerLab/FIt-SNE
# Then you NEED to execute
# g++ -std=c++11 -O3  src/sptree.cpp src/tsne.cpp src/nbodyfft.cpp  -o bin/fast_tsne -pthread -lfftw3 -lm
# Then you NEED to add the repository path (which will have your compiled code in ./bin)
# to sys as shown on the next lines

fast = True
if fast:
    sys.path.append('C:/users/jtay/Desktop/6242/viz proto/bin')
    sys.path.append('C:/users/jtay/Desktop/6242/viz proto')
    from fast_tsne import fast_tsne # O(N) via FFT, see all the comments above...
else:
    from sklearn.manifold import TSNE # O(Nlog(N))

In [70]:
RAND = 4
workers = os.cpu_count() - 2

In [71]:
MOVIE_ID = 'movieId'
TITLE = 'title'
RATING = 'rating'
VECTOR = 'vector'
GENRES = 'genres'
MEAN = 'mean'
COUNT = 'count'
STDDEV = 'std'
X = 'x'
Y = 'y'
CLUSTER = 'cluster'
COLOR = 'color'

In [72]:
model_filename = 'w2v_vs_64_sg_1_hs_1_mc_1_it_4_wn_32_ng_2_all_data_trg_val_tst.gensim'
model = Word2Vec.load(os.path.join('./gensim_models2', model_filename))

In [73]:
with open('metadata.pkl', 'rb') as f:
    dict_metadata = pickle.load(f)

In [74]:
dict_metadata[1]

{'Title': 'Toy Story',
 'Year': '1995',
 'Rated': 'G',
 'Released': '22 Nov 1995',
 'Runtime': '81 min',
 'Genre': 'Animation, Adventure, Comedy, Family, Fantasy',
 'Director': 'John Lasseter',
 'Writer': 'John Lasseter (original story by), Pete Docter (original story by), Andrew Stanton (original story by), Joe Ranft (original story by), Joss Whedon (screenplay by), Andrew Stanton (screenplay by), Joel Cohen (screenplay by), Alec Sokolow (screenplay by)',
 'Actors': 'Tom Hanks, Tim Allen, Don Rickles, Jim Varney',
 'Plot': 'A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that he has life (as a toy) good. However, he must worry about Andy\'s family moving, and what Woody does not know is about Andy\'s birthday party. Woody does not realize that Andy\'s mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a

In [75]:
df_movies = pd.read_csv('ml-20m/movies.csv', index_col=MOVIE_ID)

In [76]:
df_rating = pd.read_csv('ml-20m/ratings.csv')
df_rating = df_rating.groupby([MOVIE_ID])[RATING].agg(['count', 'mean', 'std'])

In [77]:
def get_movie_vector(i):
    try:
        return model.wv.get_vector(str(i))
    except KeyError:
        return np.nan

In [78]:
df_movies[VECTOR] = df_movies.index.get_level_values(MOVIE_ID).map(get_movie_vector)

In [79]:
len(df_movies)

27278

In [80]:
df_movies = df_movies[pd.notnull(df_movies[VECTOR])].copy()

In [81]:
len(df_movies)

23892

In [82]:
vectors = df_movies[VECTOR].to_numpy()

In [83]:
vectors = np.vstack(vectors)

In [84]:
num_clusters = [50,500,1500,5000,10000,len(vectors)]
cluster_list = []
for k in num_clusters:
    clusterer = AgglomerativeClustering(n_clusters=k,linkage='ward',)
    clusterer = clusterer.fit(vectors)
    cluster_list.append(clusterer)
    print(f'k={k} done')

k=50 done
k=500 done
k=1500 done
k=5000 done
k=10000 done
k=23892 done


In [85]:
if fast:
    # strongly recommended, fast O(N)
    tsne_result = fast_tsne(vectors, seed=RAND, nthreads=workers)
else:
    # never finished, slow O(Nlog(N))
    tsne_result = TSNE(vectors, random_state=RAND)

In [86]:
df_movies[X] = tsne_result[:, 0]
df_movies[Y] = tsne_result[:, 1]

In [87]:
# palette = palettes.Category20_20

In [88]:
print(len(cluster_list))
for clusterer in cluster_list:
    print(clusterer.children_.shape)

6
(23891, 2)
(23891, 2)
(23891, 2)
(23891, 2)
(23891, 2)
(23891, 2)


In [89]:
len(df_movies)

23892

In [90]:
# df_movies = bak.copy()
for i,clusterer in enumerate(cluster_list):
    df_movies[f'L{i}'] =clusterer.labels_
bak = df_movies.copy()

In [91]:
# source = ColumnDataSource(df_movies)
# TOOLTIPS = [(i, '@{}'.format(i)) for i in [MOVIE_ID, TITLE, X, Y]]
# fig = figure(plot_width=800, plot_height=800, tooltips=TOOLTIPS, output_backend='webgl')
# fig.circle('x', 'y', source=source, size=8, color=COLOR)

for level in [f'L{i}' for i in range(6)]:
    df_movies[level+'x'] = df_movies.groupby(level)['x'].transform('mean')
    df_movies[level+'y'] = df_movies.groupby(level)['y'].transform('mean')

# print(df_movies.head()['L0x'])
for level in [f'L{i}' for i in range(6)]:
    del df_movies[level]

In [92]:
# output_file('bokeh_{}.html'.format(model_filename), title=model_filename, mode='cdn')

In [93]:
# show(fig)

In [94]:
del df_movies['x']
del df_movies['y']
print(df_rating.head())
print(df_movies.head())


         count      mean       std
movieId                           
1        49695  3.921240  0.889012
2        22243  3.211977  0.951150
3        12735  3.151040  1.006642
4         2756  2.861393  1.095702
5        12161  3.064592  0.982140
                                      title  \
movieId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                              genres  \
movieId                                                
1        Adventure|Animation|Children|Comedy|Fantasy   
2                         Adventure|Children|Fantasy   
3                                     Comedy|Romance   
4                               Comedy|Drama|Romance   
5                                             Comedy   

                                

In [95]:
df_movies = df_movies.merge(df_rating, on=[MOVIE_ID])
print(df_movies.head())
# df_movies=df_movies.rename(columns={'count_x':'count'})

                                      title  \
movieId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                              genres  \
movieId                                                
1        Adventure|Animation|Children|Comedy|Fantasy   
2                         Adventure|Children|Fantasy   
3                                     Comedy|Romance   
4                               Comedy|Drama|Romance   
5                                             Comedy   

                                                    vector        L0x  \
movieId                                                                 
1        [0.059872735, -0.36095208, -0.077433385, 0.011... -25.121387   
2        [0.1691944, -0.062943615, -0.10992637, -0.4388...

In [96]:
# cols = [TITLE, GENRES, X, Y,] + cluster_names + [ MEAN, STDDEV, COUNT]

del df_movies['vector']

df_movies.to_csv(
    'Movie_Data_{}.tsv'.format(model_filename), sep='\t')

In [102]:
# This is used to write data to Django's database
df_output = df_movies.copy()
df_output = df_output.rename(columns={
    'title': 'movie_title',
})
df_output.index.rename('movie_id', inplace=True)
df_output['embedder'] = model_filename

In [103]:
# sorted(list(df_output[POSTER_URL].unique()))

In [104]:
# Metadata
POSTER_URL = 'poster_url'
RUNTIME = 'runtime'
DIRECTOR = 'director'
ACTORS = 'actors'
METASCORE = 'metascore'
IMDB_RATING = 'imdb_rating'
IMDB_VOTES = 'imdb_votes'

df_output[POSTER_URL] = df_output.index.map(lambda x: dict_metadata[x]['Poster']).map(
    lambda x: None if x == 'N/A' else x)
df_output[RUNTIME] = df_output.index.map(
    lambda x: dict_metadata[x]['Runtime']).map(
    lambda x: x.replace(' min', '')).map(
    lambda x: int(x) if x.isdigit() else None)
df_output[DIRECTOR] = df_output.index.map(lambda x: dict_metadata[x]['Director']).map(
lambda x: '|'.join(x.split(', ')))
df_output[ACTORS] = df_output.index.map(lambda x: dict_metadata[x]['Actors']).map(
lambda x: x.replace(', ', '|'))
df_output[METASCORE] = df_output.index.map(lambda x: dict_metadata[x]['Metascore']).map(
    lambda x: int(x) if x.isdigit() else None)
df_output[IMDB_RATING] = df_output.index.map(lambda x: dict_metadata[x]['imdbRating']).map(
    lambda x: float(x) if x != 'N/A' else None)
df_output[IMDB_VOTES] = df_output.index.map(lambda x: dict_metadata[x]['imdbVotes']).map(
    lambda x: int(x.replace(',', '')) if x != 'N/A' else None)

In [108]:
eng = create_engine('sqlite:///cse6242_team5/db.sqlite3')
with eng.begin() as con:
    df_output.to_sql('movie_edge_movie', con, if_exists='append')

In [106]:
df_output.columns

Index(['movie_title', 'genres', 'L0x', 'L0y', 'L1x', 'L1y', 'L2x', 'L2y',
       'L3x', 'L3y', 'L4x', 'L4y', 'L5x', 'L5y', 'count', 'mean', 'std',
       'embedder', 'poster_url', 'runtime', 'director', 'actors', 'metascore',
       'imdb_rating', 'imdb_votes'],
      dtype='object')

In [107]:
df_output.head()


,movie_title,genres,L0x,L0y,L1x,L1y,L2x,L2y,L3x,L3y,...,mean,std,embedder,poster_url,runtime,director,actors,metascore,imdb_rating,imdb_votes
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,-25.121387,-14.457901,-26.261216,-17.192878,-24.556549,-19.140869,-26.546903,-26.316064,...,3.921240,0.889012,w2v_vs_64_sg_1_hs_1_mc_1_it_4_wn_32_ng_2_all_d...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...,81.0,John Lasseter,Tom Hanks|Tim Allen|Don Rickles|Jim Varney,95.0,8.3,810875.0
2,Jumanji (1995),Adventure|Children|Fantasy,-24.079591,-40.059068,-23.398433,-37.803330,-23.793513,-38.014708,-23.621806,-37.434528,...,3.211977,0.951150,w2v_vs_64_sg_1_hs_1_mc_1_it_4_wn_32_ng_2_all_d...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...,104.0,Joe Johnston,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,39.0,7.0,274701.0
3,Grumpier Old Men (1995),Comedy|Romance,-19.071078,-33.318404,-19.104410,-34.704931,-19.565281,-36.100801,-19.259646,-35.792775,...,3.151040,1.006642,w2v_vs_64_sg_1_hs_1_mc_1_it_4_wn_32_ng_2_all_d...,https://m.media-amazon.com/images/M/MV5BMjQxM2...,101.0,Howard Deutch,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,46.0,6.6,22581.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,-24.079591,-40.059068,-26.348962,-42.118087,-25.380020,-41.287062,-25.380020,-41.287062,...,2.861393,1.095702,w2v_vs_64_sg_1_hs_1_mc_1_it_4_wn_32_ng_2_all_d...,https://m.media-amazon.com/images/M/MV5BYzcyMD...,124.0,Forest Whitaker,Whitney Houston|Angela Bassett|Loretta Devine|...,NaN,5.9,8833.0
5,Father of the Bride Part II (1995),Comedy,-19.071078,-33.318404,-19.104410,-34.704931,-19.565281,-36.100801,-19.259646,-35.792775,...,3.064592,0.982140,w2v_vs_64_sg_1_hs_1_mc_1_it_4_wn_32_ng_2_all_d...,https://m.media-amazon.com/images/M/MV5BOTEyNz...,106.0,Charles Shyer,Steve Martin|Diane Keaton|Martin Short|Kimberl...,49.0,6.0,31738.0
